In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from datetime import timedelta
from src.constants import string_date_format, extreme_threshold, breakout_threshold
from src.market_profile import calculate_momentum_ranks, scan_momentum_ranks, plot_scan_results
from src.utils import get_market_hours, get_last_market_day, import_watchlist
from src.pull_data import pull_watchlist_data

# Testing

In [3]:
watchlist = import_watchlist(os.path.abspath(r"files/watchlist.txt"))
last_market_day = get_last_market_day()
starting_day = last_market_day - timedelta(days=30)
df_raw = pull_watchlist_data(watchlist, start=starting_day, end=last_market_day+timedelta(days=1))

[*********************100%***********************]  89 of 89 completed


In [ ]:
def calculate_momentum_ranks(df:pd.DataFrame) -> pd.DataFrame:
    results = {}
    available_dates = pd.Series(df.index.date).sort_values(ascending=True).unique().tolist()
    market_hours = {
        date: get_market_hours(date)
        for date in available_dates
    }

    for symbol in df_raw.columns.levels[0]:
        results[symbol] = {}
        df_symbol = df[symbol].copy()

        df_symbol['First Hour'] = False
        df_symbol['Last Half Hour'] = False

        for date in available_dates:
            opening_bell, closing_bell = market_hours[date]
            first_30 = opening_bell
            second_30 = opening_bell + timedelta(minutes=30)
            last_30 = closing_bell - timedelta(minutes=30)
            if first_30 in df_symbol.index:
                df_symbol.at[first_30, 'First Hour'] = True
            if second_30 in df_symbol.index:
                df_symbol.at[second_30, 'First Hour'] = True
            if last_30 in df_symbol.index:
                df_symbol.at[last_30, 'Last Half Hour'] = True
        value_areas = {}
        
        for date in available_dates:
            str_date = date.strftime(string_date_format)
            prev_date = (date - timedelta(days=1)).strftime(string_date_format)

            tpo = calculate_tpo(df_symbol, str_date)
            value_areas[str_date] = calculate_value_area(tpo)

            if prev_date in value_areas:
                df_session = df_symbol.loc[str_date].copy()
                
                opening, extension, closing = rank_momentum(
                    df_session,
                    value_areas[str_date],
                    value_areas[prev_date]
                )
                results[symbol][str_date] = {
                    'Opening Rank': opening,
                    'Extension Rank': extension,
                    'Closing Rank': closing,
                    'Total Rank': opening + extension + closing
                }
                # results[symbol][current_date]['Value Area'] = value_areas[current_date]

    return pd.concat({
        symbol: pd.DataFrame.from_dict(inner_dict, orient='index')
        for symbol, inner_dict in results.items()
    }, axis=1)

def scan_momentum_ranks(df, extreme_threshold:float=extreme_threshold, breakout_threshold:float=breakout_threshold) -> pd.DataFrame:
    signals = dict()
    for symbol in watchlist:
        symbol_rank = df[symbol]['Total Rank']
        last_rank = symbol_rank.iloc[-1]
        if (last_rank >= extreme_threshold):
            signals[f"{symbol}:bull"] = df[symbol]['Total Rank']
        elif (last_rank <= -extreme_threshold):
            signals[f"{symbol}:bear"] = df[symbol]['Total Rank']
        elif symbol_rank.iloc[-3:].between(-breakout_threshold, breakout_threshold).all():
            signals[f"{symbol}:breakout"] = df[symbol]['Total Rank']

    return pd.DataFrame(signals)

def plot_scan_results(df:pd.DataFrame, ):
    for symbol in df.columns:
        fig = px.bar(df, x=df.index, y=symbol, title=symbol, text_auto=True)
        fig.add_hline(y=5, line_color='blue', line_dash='dot', line_width=0.5)
        fig.add_hline(y=-5, line_color='blue', line_dash='dot', line_width=0.5)
        fig.add_hline(y=9, line_color='red', line_dash='dot', line_width=0.5)
        fig.add_hline(y=-9, line_color='red', line_dash='dot', line_width=0.5)
        fig.update_yaxes(range=[-11,11])
        fig.show()

In [5]:
momentum = calculate_momentum_ranks(df_raw)

Index(['AAPL', 'ABBV', 'ACN', 'ADBE', 'ADI', 'AMAT', 'AMD', 'AMZN', 'AVGO',
       'AXP', 'BA', 'BAC', 'C', 'CAT', 'CCL', 'COP', 'COST', 'CRM', 'CSCO',
       'CVX', 'DAL', 'DE', 'DIA', 'DIS', 'FDX', 'GE', 'GLD', 'GOOG', 'GOOGL',
       'GS', 'HD', 'HON', 'IBM', 'INTC', 'IWM', 'IYR', 'JNJ', 'JPM', 'KO',
       'KRE', 'LLY', 'LQD', 'MA', 'MCD', 'META', 'MS', 'MSFT', 'MU', 'NFLX',
       'NKE', 'NOW', 'NVDA', 'OIH', 'ORCL', 'OXY', 'PEP', 'PG', 'PLTR', 'PYPL',
       'QQQ', 'RBLX', 'RTX', 'SBUX', 'SLV', 'SMH', 'SOFI', 'SOUN', 'SPY',
       'TGT', 'TLT', 'TMUS', 'TSLA', 'TSM', 'TXN', 'UAL', 'UBER', 'UNH', 'UPS',
       'V', 'WFC', 'WMT', 'XHB', 'XLE', 'XLI', 'XLK', 'XLV', 'XOM', 'XOP',
       'XYZ'],
      dtype='object')

In [6]:
scan = scan_momentum_ranks(momentum)

In [10]:
plot_scan_results(scan)

In [7]:
# Find stocks with >= 3 sessions between +/- 5 ==> watch for breakout
# Find stocks with >= 3 sessions above or below +/- 9 ==> watch for exhaustion

In [8]:

for symbol in scan.columns:
    fig = px.bar(scan, x=scan.index, y=symbol, title=symbol, text_auto=True)
    fig.add_hline(y=5, line_color='blue', line_dash='dot', line_width=0.5)
    fig.add_hline(y=-5, line_color='blue', line_dash='dot', line_width=0.5)
    fig.add_hline(y=9, line_color='red', line_dash='dot', line_width=0.5)
    fig.add_hline(y=-9, line_color='red', line_dash='dot', line_width=0.5)
    fig.update_yaxes(range=[-11,11])
    fig.show()